In [261]:
import datetime
import csv

In [262]:
import pymongo
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
from pprint import pprint
# connect to MongoDB, change the << MONGODB URL >> to reflect your own connection string
#client = MongoClient('localhost', 27017)
#db=client.admin
# Issue the serverStatus command and print the results
#serverStatusResult=db.command("serverStatus")

In [263]:
#client = MongoClient('localhost',27017)

In [264]:
#db = client.CQT

In [265]:
#db.collection_names()

In [266]:
class parameter:
    def __init__(self,pname,idx=None,node=None):
        self.name = pname
        self.Ts = None
        self.Vals = None
        self.idx  = idx
        self.node = node
    def getdata(self,db,Ts_conditions, idx=None, node=None):
        conditions = []
        if self.name is None:
            print ("parameter missing in getdata")
            return
        
        conditions.append({'Param.Name':str(self.name)})
        if self.idx is not None:
            conditions.append({'Param.Index': int(self.idx)})
        if self.node is not None:
            conditions.append( {'Param.Node': int(self.node)})
        
        conditions.append({'Ts':Ts_conditions})
        results = db.ParamData.find( {'$and':conditions} ).sort('Ts',pymongo.DESCENDING)
        
        self.Ts = []
        self.Vals = []
        for x in results:   
            #[TODO] compare with the last value, if the jump is too big ignor, otherwise include. 
            #if x['Val'] == 0:
            #    continue 
            self.Ts.append(x['Ts'])
            self.Vals.append(x['Val'])
    
    def status (self):
        print (self.name, self.idx,self.node, "len(Ts)=",len(self.Ts),"len(Vals)=", len(self.Vals))


In [267]:
class Eps:
    
    def __init__(self, dbname = 'CQT', start_time=None, end_time=None):
        
        self.dbname = dbname
        self.st = start_time
        self.et = end_time
        
        self.client = None
        self.db = None

        self.Ts_conditions = {}
        self.common_conditions = {}
        
        self.params = {}
        
        self.params['vbatt'] = parameter('vbatt')
        
        self.params['temp_0'] =  parameter('temp',idx=0)
        
        self.params['temp_1'] =  parameter('temp',idx=1)
        self.params['temp_2'] =  parameter('temp',idx=2)
        
        self.params['temp_3'] =  parameter('temp',idx=3)
        
        self.params['temp_4'] =  parameter('temp',idx=4)
        self.params['temp_5'] =  parameter('temp',idx=5)
        
        # Solar power input
        self.params['Vin_0'] = parameter('vboost',idx=0,node=2) #node = 2 by default here
        self.params['Vin_1'] = parameter('vboost',idx=1,node=2)
        self.params['Vin_2'] = parameter('vboost',idx=2,node=2)
        self.params['Cin_0'] = parameter('curin',idx=0,node=2) 
        self.params['Cin_1'] = parameter('curin',idx=1,node=2)
        self.params['Cin_2'] = parameter('curin',idx=2,node=2)     
        
        # Power output
        self.params['V_obc'] = parameter('out_val',idx=0,node=2) 
        self.params['V_radio'] = parameter('out_val',idx=3,node=2)
        self.params['V_payload'] = parameter('out_val',idx=5,node=2)
        self.params['C_obc'] = parameter('curout',idx=0,node=2) 
        self.params['C_radio'] = parameter('curout',idx=3,node=2)
        self.params['C_payload'] = parameter('curout',idx=5,node=2)
        
        self.params['battmode'] = parameter('battmode')
        #self.params['crusun'] = parameter('crusun')
        #self.params['crusys'] = parameter('crusys')

        # Power output
        
        
        

        
        if self.st is not None:
            self.Ts_conditions['$gte'] = int(self.st)
        else:
            self.Ts_conditions['$gte'] = int(1452160013) #GMT: Thursday, January 7, 2016 9:46:53 AM
        if self.et is not None:
            self.Ts_conditions['$lt'] = int(self.et)
    
    #def __del__(self): 
    #    self.client.close()
        
    def connect (self):
        self.client = MongoClient('localhost',27017)
        self.db = self.client[self.dbname]
    
    
    def close(self):
        self.client.close()
        
    def reload(self):
        if self.db is None:
            self.connect()
        
        
        for key, value in self.params.items():
            value.getdata(self.db,self.Ts_conditions)
            
    '''
        self.getdata(self.temp_0)  # Temperature on EPS  
        self.getdata(self.temp_1)
        self.getdata(self.temp_2)
        self.getdata(self.temp_3)
        self.getdata(self.temp_4)  # Temperature on BP4  
        self.getdata(self.temp_5)
        self.getdata(self.Vin_0)   # Voltage(solar) inputs
        self.getdata(self.Vin_1)
        self.getdata(self.Vin_2)
        self.getdata(self.Cin_0)   # Current(solar) inputs
        self.getdata(self.Cin_1)
        self.getdata(self.Cin_2)
     
    
    def getdata(self,param, idx=None, node=None):
        conditions = []
        if param.name is None:
            print ("parameter missing in getdata")
            return
        
        conditions.append({'Param.Name':str(param.name)})
        if param.idx is not None:
            conditions.append({'Param.Index': int(param.idx)})
        if param.node is not None:
            conditions.append( {'Param.Node': int(param.node)})
        
        conditions.append({'Ts':self.Ts_conditions})
        results = self.db.ParamData.find( {'$and':conditions} ).sort('Ts',pymongo.DESCENDING)
        
        param.Ts = []
        param.Vals = []
        for x in results:   
            #[TODO] compare with the last value, if the jump is too big ignor, otherwise include. 
            #if x['Val'] == 0:
            #    continue 
            param.Ts.append(x['Ts'])
            param.Vals.append(x['Val'])

    '''

In [268]:
eps = Eps()

In [269]:
eps.reload()

In [270]:
for pname,param in eps.params.items():
    print (pname)
    param.status()

temp_5
temp 5 None len(Ts)= 2597 len(Vals)= 2597
C_obc
curout 0 2 len(Ts)= 2597 len(Vals)= 2597
Cin_1
curin 1 2 len(Ts)= 2597 len(Vals)= 2597
battmode
battmode None None len(Ts)= 2597 len(Vals)= 2597
Vin_0
vboost 0 2 len(Ts)= 2597 len(Vals)= 2597
C_payload
curout 5 2 len(Ts)= 2597 len(Vals)= 2597
C_radio
curout 3 2 len(Ts)= 2597 len(Vals)= 2597
temp_0
temp 0 None len(Ts)= 2597 len(Vals)= 2597
Cin_2
curin 2 2 len(Ts)= 2597 len(Vals)= 2597
temp_1
temp 1 None len(Ts)= 2597 len(Vals)= 2597
temp_3
temp 3 None len(Ts)= 2597 len(Vals)= 2597
vbatt
vbatt None None len(Ts)= 2597 len(Vals)= 2597
temp_2
temp 2 None len(Ts)= 2597 len(Vals)= 2597
Vin_1
vboost 1 2 len(Ts)= 2597 len(Vals)= 2597
Cin_0
curin 0 2 len(Ts)= 5194 len(Vals)= 5194
V_payload
out_val 5 2 len(Ts)= 2315 len(Vals)= 2315
V_radio
out_val 3 2 len(Ts)= 2315 len(Vals)= 2315
Vin_2
vboost 2 2 len(Ts)= 2597 len(Vals)= 2597
temp_4
temp 4 None len(Ts)= 2597 len(Vals)= 2597
V_obc
out_val 0 2 len(Ts)= 2315 len(Vals)= 2315
